In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
from tensorflow.python.framework import ops
import tflearn
import random

2023-02-25 16:45:21.386115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
!pip install tflearn

In [5]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
words=[]
documents=[]
classes=[]
ignore_words=['?']

stemmer = LancasterStemmer()


for intent in intents['intents']:
    for pattern in intent['patterns']:
         w=nltk.word_tokenize(pattern)
         words.extend(w)
         documents.append((w, intent['tag']))
         if intent['tag'] not in classes:
             classes.append(intent['tag'])
        

words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))


classes=sorted(list(set(classes)))

In [8]:
print (len(documents), "documents",documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)


30 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Namaste'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['TELL', 'ME', 'ABOUT', 'TAKING', 'SOIL', 'SAMPLE'], 'Soil Testing'), (['How', 'to', 'take', 'soil', 'sample', '?'], 'Soil Testing'), (['What', 'is', 'the', 'technique', 'used', 'in', 'taking', 'soil', 'sample'], 'Soil Testing'), (['TELL', 'ME', 'SOMETHING', 'ABOUT', 'NUTRIENT', 'MANAGEMENT', '?'], 'Nutrient Management'), (['What', 'are', 'some', 'ways', 'to', 'improve', 'nutrients', 'in', 'soil'], 'Nutrient Management'), (['How', 'to', 'manage', 'nutrients'], 'Nutrient Management'), (['Please', 'refer', 'the', 'following', 'website', '(', 'https', ':', '//www.epa.gov/agriculture/agriculture-nutrient-management

In [9]:

training=[]
output=[]

output_empty=[0]*len(classes)

for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
random.shuffle(training)
training=np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])


/var/folders/f8/pr3njbwj3vl3_thlf_2n0njr0000gp/T/ipykernel_76453/45723145.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [10]:
ops.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.79449 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.79449 - acc: 0.9307 -- iter: 24/30
Training Step: 4000  | total loss: 0.71810 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.71810 - acc: 0.9376 -- iter: 30/30
--
INFO:tensorflow:/Users/aditya/Downloads/Farmer-friendly-contextual-chatbot-master/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open('training_data','wb'))